***
## Algorithmic fairness with AI Fairness 360 (IBM)
### This notebook is accompanied by a Medium article I wrote, published [here](https://medium.com/@alinedintino/mitigating-bias-and-ensuring-fairness-in-machine-learning-algorithms-a0b77a8f49eb)
***

In [1]:
# Load all necessary packages
import pandas as pd
import aif360
from aif360.datasets import GermanDataset
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.metrics import BinaryLabelDatasetMetric
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Read the GermanDataset
german_data = GermanDataset()
german_df = german_data.convert_to_dataframe()
german_df = german_df[0]
german_df.head()

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,status=A13,status=A14,credit_history=A30,credit_history=A31,credit_history=A32,credit_history=A33,credit_history=A34,purpose=A40,purpose=A41,purpose=A410,purpose=A42,purpose=A43,purpose=A44,purpose=A45,purpose=A46,purpose=A48,purpose=A49,savings=A61,savings=A62,savings=A63,savings=A64,savings=A65,employment=A71,employment=A72,employment=A73,employment=A74,employment=A75,other_debtors=A101,other_debtors=A102,other_debtors=A103,property=A121,property=A122,property=A123,property=A124,installment_plans=A141,installment_plans=A142,installment_plans=A143,housing=A151,housing=A152,housing=A153,skill_level=A171,skill_level=A172,skill_level=A173,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,credit
0,6.0,1169.0,4.0,4.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
1,48.0,5951.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0
2,12.0,2096.0,2.0,3.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,42.0,7882.0,2.0,4.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,24.0,4870.0,3.0,4.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0


In [3]:
# Define the privileged and unprivileged groups
privileged_groups = [{'sex': 1,'age': 1}]
unprivileged_groups = [{'sex': 0,'age': 0}] 

#privileged_groups = [{'sex': 1}]
#unprivileged_groups = [{'sex': 0}]

In [4]:
# Fairness performance of datasets before classification
# Constucting two functions to call the desired metrics
metric_orig = BinaryLabelDatasetMetric(german_data, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

print("Disparate impact (of original labels) between unprivileged and privileged groups = %f" % metric_orig.disparate_impact())
print("Difference in statistical parity (of original labels) between unprivileged and privileged groups = %f" % metric_orig.statistical_parity_difference())
print("Individual fairness metric that measures how similar the labels are for similar instances = %f" % metric_orig.consistency())

Disparate impact (of original labels) between unprivileged and privileged groups = 0.747630
Difference in statistical parity (of original labels) between unprivileged and privileged groups = -0.186462
Individual fairness metric that measures how similar the labels are for similar instances = 0.681600


In [5]:
# Baseline Method 
dataset_orig_train, dataset_orig_test = german_data.split([0.7], shuffle=True)

# Using Learning fair representations (LFR) to transform german dataset
# Required Inputs:
# Input recontruction quality - Ax
# Fairness constraint - Az
# Output prediction error - Ay

# Scaling the dataset (scaled dataset together with its labels is needed)
scale_orig = StandardScaler()
dataset_orig_train.features = scale_orig.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = scale_orig.transform(dataset_orig_test.features)

# LFR itself contains logistic regression sync, it uses sigmoid functions 
LFR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=5, Ax=0.1, Ay=1.0, Az=100.0, verbose=1)

TR = LFR.fit(dataset_orig_train, maxiter=5000, maxfun=5000)

# Transform training data and align features
dataset_transf_train = TR.transform(dataset_orig_train)

step: 0, loss: 2.963634337314576, L_x: 2.5346566554093064,  L_y: 0.9067669031489546,  L_z: 0.018034017686246913
step: 250, loss: 2.9636344073686245, L_x: 2.5346566987843238,  L_y: 0.9067669430305679,  L_z: 0.01803401794459624
step: 500, loss: 1.1706940521058324, L_x: 2.5402542723784203,  L_y: 0.3581552322727513,  L_z: 0.00558513392595239
step: 750, loss: 0.9420320974136516, L_x: 2.540430188565174,  L_y: 0.1550320667578237,  L_z: 0.0053295701179931045
step: 1000, loss: 0.6659995683788107, L_x: 2.5414980142049233,  L_y: -0.07263891355174788,  L_z: 0.004844886805100662
step: 1250, loss: 0.21060397314217416, L_x: 2.542904017329769,  L_y: -0.6581765260467153,  L_z: 0.006144900974559125
step: 1500, loss: -0.09507425696286814, L_x: 2.542694942480411,  L_y: -0.797484418211194,  L_z: 0.004481406670002847
step: 1750, loss: -0.09507465451654312, L_x: 2.5426949875874882,  L_y: -0.7974842905855608,  L_z: 0.004481401373102689
step: 2000, loss: -0.4248344053658831, L_x: 2.5424705616201573,  L_y: -0.9

In [6]:
# Check if the labels are transformed: counts the num. of transformed labels
k=0
for i in range(len(dataset_orig_train.labels)):
    if(dataset_transf_train.labels[i] == dataset_orig_train.labels[i]):
        pass
    else:
        k+=1
        
k

216

In [7]:
# Measuring fairness performance again
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups)

print("Disparate impact ratio (of transformed labels) between unprivileged and privileged groups = %f" % metric_transf_train.disparate_impact())
print("Difference in statistical parity (of transformed labels) between unprivileged and privileged groups = %f" % metric_transf_train.statistical_parity_difference())
print("Individual fairness metric 'consistency' that measures how similar the labels are for similar instances = %f" % metric_transf_train.consistency())

Disparate impact ratio (of transformed labels) between unprivileged and privileged groups = 1.000000
Difference in statistical parity (of transformed labels) between unprivileged and privileged groups = 0.000000
Individual fairness metric 'consistency' that measures how similar the labels are for similar instances = 1.000000
